In [10]:
!pip install ultralytics
!pip install -U ipywidgets
from ultralytics import YOLO
import torch

^C
ERROR: Operation cancelled by user


UnboundLocalError: local variable 'child' referenced before assignment

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data='/kaggle/input/cv-project-files2/dataset.yaml',
    epochs=75,                            # Number of training epochs
    imgsz=640,                            # Image size
    batch=32,                             # batch size
    lr0=0.001,                             # initial learning rate
    augment=True,                         # no data augmentation
    dropout=0.2       ,                    # dropout to reduce overfitting
    device=[0,1]
)


In [ ]:
#from ultralytics import YOLO
# resume training
#model = YOLO('/kaggle/working/runs/detect/train/weights/last.pt')
#model.train(resume=True)

In [ ]:
image_folder = '/kaggle/input/large-license-plate-dataset/images/test/'
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

results = model(image_folder,stream=False)
output_folder = '/kaggle/working/test'
ok = 0
skip = 0
for image in results:
    device = image.boxes.xywh.device
    suitable_crops = torch.empty((0,6), device=device)
    for i in range(len(image)):
        x, y, w, h = image.boxes.xywh[i].tolist() 
        if w > 50 and h > 20:
            conf = image.boxes.conf[i]
            keep_crop = torch.tensor([[x-w/2,y-h/2,x+w/2,y+h/2,conf,0]],device=device)
            suitable_crops = torch.cat((suitable_crops,keep_crop))
            ok+=1
        else:
            skip+=1
    image.update(suitable_crops)
    image.save_crop(output_folder)

print("ok",ok,"skip",skip)

In [ ]:
# to save the run : zip the folder from this command and then download the zip file
#import shutil
#shutil.make_archive("train_20", 'zip', "/kaggle/working/runs/detect/train")

## PaddleOCR

In [ ]:
!pip install paddlepaddle-gpu
!pip install paddleocr

In [ ]:
import pandas as pd
import os
from paddleocr import PaddleOCR
from PIL import Image
import pytesseract
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [77]:
def process_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #display_image(image, title="Original Image")
    #display_image(gray_image, title="Grayscale Image")
    
    gaussian_image = cv2.GaussianBlur(gray_image, (3, 3), 0)
    #display_image(gaussian_image, title="Gaussian Blur")
    
    process_path = '/kaggle/working/PaddleOCR/preprocessed_license_plate.jpg'
    cv2.imwrite(process_path, gaussian_image)
    

def display_image(image, title="Image", cmap='gray'):
    plt.figure(figsize=(2, 2))  
    plt.imshow(image, cmap=cmap) 
    plt.title(title) 
    plt.axis('off')
    plt.show()

In [78]:
csv_path = '/kaggle/input/license-plate-text-recognition-dataset/lpr.csv'  
images_folder = '/kaggle/input/license-plate-text-recognition-dataset/cropped_lps/cropped_lps'  
process_path = '/kaggle/working/PaddleOCR/preprocessed_license_plate.jpg'
alphabet_path = '/kaggle/input/cv-project-files2/alphabet.txt'

ocr = PaddleOCR(use_gpu=True,lang="en",show_log=False, use_angle_cls=True)

data = pd.read_csv(csv_path)

[2024/11/28 16:55:25] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/28 16:55:26] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/28 16:55:27] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0


In [80]:
correct_count = 0
recognized_count = 0
total_count = 0
almost_count = 0
i = 0
for index, row in data.iterrows():    
    if i < 5000:
        i+=1
        image_path = os.path.join(images_folder, row['images'])
        ground_truth = row['labels']
        image = cv2.imread(image_path)
        
        process_image(image)
        result = ocr.ocr(process_path, rec=True)
        if result and result[0]:
            recognized_text = result[0][0][1][0]
            recognized_text = recognized_text.replace(" ", "").replace("-", "")
            recognized_count += 1
        else:
            recognized_text = ""
        
        if recognized_text == ground_truth:
            correct_count += 1
        elif recognized_text in ground_truth and recognized_text != "":
            almost_count+=1
            
        total_count+=1
    else:
        break
        
accuracy = correct_count / total_count * 100
print("Total: ", total_count)
print(f"Accuracy: {correct_count} = {accuracy:.2f}% ")
print(f"Recognized something: {recognized_count} ")
print(f"Almost recognized: {almost_count} out of incorrects {total_count - correct_count}")

Total:  5000
Accuracy: 2529 = 50.58% 
Recognized something: 3772 
Almost recognized: 353 out of incorrects 2471
